# Blockchain Implementation in Python Programming Language

#### Importing Necessary Modules

In [15]:
import re, copy
import pandas as pd
import numpy as np
import hashlib
import pprint
pp = pprint.PrettyPrinter(indent=0)

#### Creating a Time-Stamped Transaction 

In [16]:
txn_example = pd.DataFrame([{'timestamp' : pd.Timestamp.now(),
                            'to_account' : 'Harsh',
                            'from_account' : 'Hrituraj',
                            'amount' : 3000}])

In [17]:
txn_example

,amount,from_account,timestamp,to_account
0,3000,Hrituraj,2019-10-25 16:34:21.848698,Harsh


#### Generating Transactions on the Ledger 

In [18]:
def transaction(from_account, to_account, amount):
    if isinstance(amount, int) or isinstance(amount, float):
        new_txt = pd.DataFrame([{'timestamp' : pd.Timestamp.now(),
                            'from_account' : from_account,
                            'to_account' : to_account,
                            'amount' : amount}])
        return new_txt
    else:
        print('Amount must be numeric')

In [19]:
ldgr_txn_genesis = transaction(from_account='Harikrishna', 
                               to_account = 'Hrituraj',
                               amount = 5000
                              )

In [20]:
ledger = ldgr_txn_genesis
ledger

,amount,from_account,timestamp,to_account
0,5000,Harikrishna,2019-10-25 16:34:22.299407,Hrituraj


#### Balancing the Account on Ledger 

In [21]:
def get_balance_ldgr(ledger, account):
    deposits = ledger[ledger.to_account == account]['amount']
    total_deposits = deposits.sum()
    
    withdrawls = ledger[ledger.from_account == account]['amount']
    total_withdrawls = withdrawls.sum()
    
    balance = total_deposits - total_withdrawls
    
    return balance

In [22]:
get_balance_ldgr(ldgr_txn_genesis, 'Hrituraj')

5000

In [23]:
ldgr_txn_hritu2harsh = transaction(from_account = 'Hrituraj',
                                  to_account = 'Harsh',
                                  amount = 4000
                                 )
ldgr_txn_hritu2harsh

,amount,from_account,timestamp,to_account
0,4000,Hrituraj,2019-10-25 16:34:24.175365,Harsh


In [24]:
ledger = ledger.append(ldgr_txn_hritu2harsh)
ledger

,amount,from_account,timestamp,to_account
0,5000,Harikrishna,2019-10-25 16:34:22.299407,Hrituraj
0,4000,Hrituraj,2019-10-25 16:34:24.175365,Harsh


In [25]:
for account in ['Harsh', 'Hrituraj', 'Rajput']:
    print('Balance of {}: {}'.format(account, 
                                     get_balance_ldgr(ledger, account)))

Balance of Harsh: 4000
Balance of Hrituraj: 1000
Balance of Rajput: 0


In [28]:
ldgr_txn_harsh2rajput = transaction(from_account = 'Harsh',
                                       to_account = 'Rajput',
                                       amount = 5000
                                      )

In [29]:
ledger = ledger.append(ldgr_txn_harsh2rajput)
ledger

,amount,from_account,timestamp,to_account
0,5000,Harikrishna,2019-10-25 16:34:22.299407,Hrituraj
0,4000,Hrituraj,2019-10-25 16:34:24.175365,Harsh
0,5000,Harsh,2019-10-25 16:35:55.224663,Rajput


In [30]:
for account in ['Harsh', 'Hrituraj', 'Rajput']:
    print('Balance of {}: {}'.format(account, 
                                     get_balance_ldgr(ledger, account)))

Balance of Harsh: -1000
Balance of Hrituraj: 1000
Balance of Rajput: 5000


#### Creating a Blockchain Ledger 

In [31]:
def transaction_bc(from_account, to_account, amount):
    if isinstance(amount, int) or isinstance(amount, float):
        transaction_dict = {'timestamp' : pd.Timestamp.now(),
                            'from_account' : from_account,
                            'to_account' : to_account,
                            'amount' : amount}
        return transaction_dict
    else:
        print('Amount must be numeric')

In [32]:
mayank_transaction = transaction_bc(from_account='Mayank', 
                                              to_account = 'Hrituraj',
                                              amount = 2000
                                     )

mayank_block = {'transactions' : mayank_transaction,
                 'prev_hash' : None,
                 'proof_of_work' : 0
                }

In [33]:
blockchain = [mayank_block]
pp.pprint(blockchain)

[{'prev_hash': None,
'proof_of_work': 0,
'transactions': {'amount': 2000,
                'from_account': 'Mayank',
                'timestamp': Timestamp('2019-10-25 16:38:10.690798'),
                'to_account': 'Hrituraj'}}]


#### Using Hashing Function to mask the Transactions 

In [34]:
def hash_sha256(obj):
    return hashlib.sha256(str(obj).encode('utf-8')).hexdigest()

In [35]:
mayank_hash = hash_sha256(mayank_block)
mayank_hash

'ad4e68b523581ad374171ce2af3d5c78827f73061f5b8f77be560ce9c7992219'

#### Validation Function

In [36]:
def is_valid_proof(last_proof, this_proof):
    guess = str(last_proof) + str(this_proof)
    guess_hash = hash_sha256(guess)
    is_valid = bool(re.search('0{3}$',guess_hash))
    return is_valid

#### Proof-of-Work Function 

def proof_of_work(last_proof):
    candidate_proof = 0
    while not is_valid_proof(last_proof, candidate_proof):
        candidate_proof += 1
    return candidate_proof

#### Account Balance on Blockchain

In [38]:
def get_balance(blockchain, account):
    ldgr_txns = []
    for block in blockchain:
        ldgr_txns.append(block['transactions'])
        
    balance = get_balance_ldgr(pd.DataFrame(ldgr_txns), account)
    
    return balance

In [45]:
second_proof = proof_of_work(mayank_block['proof_of_work'])
print(second_proof)

5735


In [46]:
get_balance(blockchain, 'Hrituraj')

2000

#### Creating New Block 

In [47]:
def create_block(txns, prev_hash = None, proof_of_work = None):
    new_block = {'transactions' : txns, 
                 'prev_hash' : prev_hash,
                 'proof_of_work' : proof_of_work}
    return new_block

In [48]:
txn_harsh = transaction_bc(from_account = 'Hrituraj',
                         to_account = 'Harsh',
                         amount = 900
                        )  

In [49]:
second_block = create_block(txns = txn_harsh, 
                            prev_hash = mayank_hash,
                            proof_of_work = second_proof)

In [50]:
def add_block_if_valid(blockchain, new_block):
    new_blockchain = copy.deepcopy(blockchain)
    
    check_proof = is_valid_proof(blockchain[-1]['proof_of_work'], 
                                 new_block['proof_of_work'])
    
    last_hash = hash_sha256(blockchain[-1])
    check_hash = last_hash == new_block['prev_hash']
        
    acct_balance = get_balance(blockchain, new_block['transactions']['from_account'])
    check_balance = acct_balance > new_block['transactions']['amount']
    
    if check_proof and check_hash and check_balance:
        new_blockchain.append(new_block)
    else:
        print('This block is not valid.')
        
    return new_blockchain

In [51]:
blockchain = add_block_if_valid(blockchain, second_block)
pp.pprint(blockchain)

[{'prev_hash': None,
'proof_of_work': 0,
'transactions': {'amount': 2000,
                'from_account': 'Mayank',
                'timestamp': Timestamp('2019-10-25 16:38:10.690798'),
                'to_account': 'Hrituraj'}},
{'prev_hash': 'ad4e68b523581ad374171ce2af3d5c78827f73061f5b8f77be560ce9c7992219',
'proof_of_work': 5735,
'transactions': {'amount': 900,
                'from_account': 'Hrituraj',
                'timestamp': Timestamp('2019-10-25 16:42:07.020506'),
                'to_account': 'Harsh'}}]


In [52]:
txn_harsh = transaction_bc(from_account = 'Harsh',
                        to_account = 'Harikrishna',
                        amount = 300)

In [53]:
third_block = create_block(txns = txn_harsh,
                           prev_hash = hash_sha256(blockchain[-1]),
                           proof_of_work = proof_of_work(blockchain[-1]['proof_of_work'])
                          )

In [54]:
third_block


{'transactions': {'timestamp': Timestamp('2019-10-25 16:43:08.630424'),
  'from_account': 'Harsh',
  'to_account': 'Harikrishna',
  'amount': 300},
 'prev_hash': '939cee92a8d942038dccdcc9285b829b8063cd235b6bfdec54b8a9a68fcf4026',
 'proof_of_work': 626}

In [55]:
blockchain = add_block_if_valid(blockchain, third_block)
pp.pprint(blockchain)

[{'prev_hash': None,
'proof_of_work': 0,
'transactions': {'amount': 2000,
                'from_account': 'Mayank',
                'timestamp': Timestamp('2019-10-25 16:38:10.690798'),
                'to_account': 'Hrituraj'}},
{'prev_hash': 'ad4e68b523581ad374171ce2af3d5c78827f73061f5b8f77be560ce9c7992219',
'proof_of_work': 5735,
'transactions': {'amount': 900,
                'from_account': 'Hrituraj',
                'timestamp': Timestamp('2019-10-25 16:42:07.020506'),
                'to_account': 'Harsh'}},
{'prev_hash': '939cee92a8d942038dccdcc9285b829b8063cd235b6bfdec54b8a9a68fcf4026',
'proof_of_work': 626,
'transactions': {'amount': 300,
                'from_account': 'Harsh',
                'timestamp': Timestamp('2019-10-25 16:43:08.630424'),
                'to_account': 'Harikrishna'}}]


#### FInal Balance: 

In [56]:
for account in ['Harsh', 'Hrituraj', 'Harikrishna']:
    print('Balance of {}: {}'.format(account, 
                                     get_balance(blockchain, account)))

Balance of Harsh: 600
Balance of Hrituraj: 1100
Balance of Harikrishna: 300


# END